# Lab Session 2: Longitudinal Behavioral Analysis & Mental Health (LMM)

In this session, we will link people's high-frequency digital traces with their psychological well-being. BWe will analyze how web browsing patterns correlate with **Loneliness (UCLA-3)** and **Stress (PSS-10)** using a dataset covering 50 individuals over 6 months.

To account for the **longitudinal and nested structure** of this data (repeated observations within individuals), we will implement **Linear Mixed-Effects Models (LMM)**.

### What we will do?
* **Perform Data Alignment**: Merge datasets with different temporal resolutions (e.g., second-level logs vs. monthly surveys).
* **Execute Feature Engineering**: Calculate online behavioral features such as diurnal activity patterns (Day vs. Night) and category-specific engagement.
* **Implement Mixed-Effects Modeling**: Understand the practical distinction between **Fixed Effects** (population-level trends) and **Random Effects** (individual variations).


### Pre-reading & Resources on LMM
If you need a refresher on Linear Mixed Models, please refer to:
* **The "Plain English" Guide**: [Introduction to Mixed Effects Models](https://ourcodingclub.github.io/tutorials/mixed-models/) (Coding Club)
* **The Academic Reference**: Meteyard, L., & Davies, R. A. (2020). [Best practice guidance for linear mixed-effects models in psychological science](https://pmc.ncbi.nlm.nih.gov/articles/PMC7153721/). *Journal of Memory and Language*.
* **Documentation**: [Statsmodels LMM Overview](https://www.statsmodels.org/stable/mixed_linear.html) - *Technical documentation for the library we will use today.*

## Programming Toolkit

To perform this analysis, we will use several specialized Python libraries. You can refer to the official documentation for detailed API information:

| Library | Category | Purpose | Documentation |
| :--- | :--- | :--- | :--- |
| **Pandas** | Data Manipulation | Handling dataframes, time-series alignment, and merging. | [Link](https://pandas.pydata.org/docs/) |
| **NumPy** | Numerical Computing | Vectorized operations and mathematical functions. | [Link](https://numpy.org/doc/) |
| **Statsmodels** | Statistical Modeling | Implementing Linear Mixed-Effects Models (LMM) and summary stats. | [Link](https://www.statsmodels.org/stable/index.html) |
| **Seaborn** | Data Visualization | Drawing statistical graphics. | [Link](https://seaborn.pydata.org/) |
| **Matplotlib** | Plotting Engine | Base library for plot aesthetics and layouts. | [Link](https://matplotlib.org/stable/index.html) |
| **Scipy** | Scientific Computing | Used for statistical tests and probability distributions. | [Link](https://docs.scipy.org/doc/scipy/) |

>Run the code cell below to initialize your environment.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import warnings

# Suppress warnings for cleaner output during the lab
warnings.filterwarnings('ignore')

# --- Visualization Settings ---
sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10
})

## 2. Data Acquisition & Structural Overview

In this session, we work with four datasets. Our goal is to align these into a single "longitudinal" dataframe ready for modeling.

### 2.1 Dataset Descriptions

| Dataset | Granularity | Key Columns | Description |
| :--- | :--- | :--- | :--- |
| **Browsing Logs** | High-frequency | `pid`, `used_at`,`domain`, `duration`, `wave` | Raw web traces recording every URL visited. |
| **Surveys** | Monthly (6 waves) | `pid`, `wave`, `ucla_3`, `pss_10` | Repeated mental health assessments. |
| **Demographics** | Static | `pid`, `gender`, `age_group` | Constant user characteristics. |
| **Domain Map** | Metadata | `domain`, `category` | Mapping URLs to functional categories (e.g., Social, News). |

### 2.2 Variables of Interest

#### **Mental Health Indicators (Dependent Variables)**
* **UCLA-3 (Loneliness)**: A 3-item scale measuring feelings of social isolation.
    * **Range**: 3 to 9. Higher scores indicate **greater loneliness**.
* **PSS-10 (Perceived Stress)**: A 10-item scale measuring the degree to which situations in life are appraised as stressful.
    * **Range**: 0 to 40. Higher scores indicate **higher perceived stress**.

#### **Demographic Coding**
* **Gender**: `1` = Male, `0` = Female.
* **Age Group**:
    * `1`: 18 – 30 years old
    * `2`: 31 – 60 years old
    * `3`: 60+ years old


In [ ]:
# --- Data Loading ---
BASE_URL = "https://raw.githubusercontent.com/Yajing-W/CSS_winterschool_practice/refs/heads/main/"

# Load the datasets from the remote repository
try:
    df_logs = pd.read_parquet(BASE_URL + "browsing_traces_waves.parquet")
    df_survey = pd.read_csv(BASE_URL + "surveys.csv")
    df_demo = pd.read_csv(BASE_URL + "demographics_session_2.csv")
    df_domain_map = pd.read_csv(BASE_URL + "domain_category_map.csv")

    print("All datasets loaded successfully.")
except Exception as e:
    print(f"Error loading data: {e}")


print(f"\n[Summary Statistics]")
print(f"- Browser Logs: {df_logs.shape[0]} records")
print(f"- Survey Responses: {df_survey.shape[0]} records")
print(f"- Demographic Profiles: {df_demo.shape[0]} users")

print(f"\n[Preview: Demographics (df_demo)]")
display(df_demo.head())

print(f"\n[Preview: Browser Logs (df_logs)]")
display(df_logs.head())

## 3.1 Feature Engineering: Defining Behavioral Markers

To find the association with mental health, we need to convert raw timestamped logs into meaningful behavioral indicators. For each user (`pid`) within each measurement period (`wave`), we will extract the following features:

| Feature | Definition | Behavioral Significance |
| :--- | :--- | :--- |
| **Total_Duration** | Total monthly online hours. | Overall digital engagement behavior. |
| **Daily_avg_duration** | Average daily online hours. | Daily intensity of digital usage. |
| **Day_Night_Diff** | (Duration 06:00-18:00) - (Duration 18:00-06:00). | Negative values indicate high nocturnal activity, often linked to sleep disruption and stress. |
| **Category_Hours** | Total hours spent on 'Social Media', 'Games', 'Shopping' and etc. | Distinguishes between communication, entertainment-seeking, and impulsive behaviors. |



> **Note on Scaling**: We convert the raw `duration` (seconds) into **Hours** to ensure our regression coefficients ($\beta$) are interpretable (e.g., "an increase of 1 hour is associated with X points of stress").

> Calculate these features and aggregate them at the `(pid, wave)` level.

In [ ]:
# Extract behavioral features from browser logs

# We merge logs with the domain map to identify the type of activity
df_logs_mapped = pd.merge(df_logs, df_domain_map, on=...., how=...)
# Assign 'uncategorized' to any domain not found in our mapping table
df_logs_mapped['category'] = df_logs_mapped['category'].fillna('uncategorized')

#Temporal Processing
df_logs_mapped['used_at'] = pd.to_datetime(df_logs_mapped['used_at'])
df_logs_mapped['hour'] = df_logs_mapped['used_at'].dt.hour
df_logs_mapped['is_daytime'] = df_logs_mapped['hour'].between(6, 17) # Day: 06:00-17:59

# Convert seconds to hours for better scale in modeling
total_dur = df_logs_mapped.groupby(['pid', 'wave'])['duration'].sum() / 3600

# Day-Night Duration Difference
day_night = df_logs_mapped.pivot_table(index=['pid', 'wave'],
                                       columns='is_daytime',
                                       values='duration',
                                       aggfunc='sum',
                                       fill_value=0) / 3600
day_night_diff = day_night.get(True, 0) - day_night.get(False, 0)

# We group by pid/wave/category and pivot to get hours per category
cat_hours = df_logs_mapped.groupby(['pid', 'wave', 'category'])['duration'].sum().unstack(fill_value=0) / 3600

# Construct Feature Dataframe
df_features = pd.DataFrame({
    'total_duration': total_dur,
    'day_night_diff': day_night_diff,
    'social_media_duration': cat_hours.get('social_media', 0),
    'games_duration': cat_hours.get('games', 0),
    'shopping_duration': cat_hours.get('shopping', 0)
}).reset_index()

df_features['daily_avg_duration'] = df_features['total_duration'] / 28
display(df_features.head())

## 3.2 Data Integration: Building the Longitudinal Dataset

Now that we have extracted behavioral markers from raw logs, we must align them with our mental health labels (Survey scores) and user characteristics (Demographics).

### The Merging Logic
To perform a **Linear Mixed-Effects (LMM)** analysis, our data must be in **Long Format**. This means:
1.  **Time-Varying Data**: Behavioral features and Survey scores are matched by both User ID (`pid`) and Time (`wave`).
2.  **Static Data**: Demographic info (Gender, Age) is matched only by User ID (`pid`) and is repeated for every wave of that user.



### Handling Missingness
Unlike traditional OLS or ANOVA, LMM does not require every participant to have data for every single wave.
However, we still need to ensure that for any given "User-Wave" observation, we have both the predictor (behavior) and the outcome (survey score).
* We use an **Inner Join** between surveys and behavior to ensure our model only trains on complete observations (Timepoints where we have both a behavioral "predictor" and a mental health "outcome").


> 1. Check for missing values in the survey dataset.
> 2. Consolidate the data into a single long-format dataframe.

In [ ]:
# Check for missing values in Survey data ---
print("Checking for missing values in Surveys:")
print(df_survey[['ucla_3', 'pss_10']].isnull().sum())

#use 'inner' join here because a model needs both X (behavior) and Y (survey)
# Merge with Demographics
df_final = .....



n_original_users = df_survey['pid'].nunique()
n_final_users = df_final['pid'].nunique()
print(f"\nOriginally: {n_original_users} users.")
print(f"After merging: {n_final_users} users retained.")
print(f"Total User-Wave observations for LMM: {len(df_final)}")

# Sort for longitudinal structure
df_final = df_final.sort_values(['pid', 'wave']).reset_index(drop=True)
display(df_final.head())

## 4. Pre-modeling Diagnostics

Before fitting our Linear Mixed-Effects Models (LMM), we must evaluate the relationships between our predictors.

### 4.1 Multicollinearity: Correlation & VIF (add some ref about this)
In behavioral research, different metrics often carry redundant information. We use two complementary tools to detect **Multicollinearity**:
1.  **Correlation Matrix (Qualitative)**: A heatmap showing pairwise relationships. We look for coefficients $|r| > 0.7$.
2.  **Variance Inflation Factor (VIF, Quantitative)**: Measures how much a variable is explained by *all other* predictors combined.
    * **VIF < 5**: Generally acceptable.
    * **VIF > 10**: Indicates high multicollinearity; the variable may need to be removed to ensure model stability.

### 4.2 Categorical Factor Conversion
We ensure `gender` and `age_group` are treated as **factors**. This allows the model to estimate differences between groups (e.g., comparing Age Group 1 vs. Group 2) rather than treating them as continuous numbers.

> **Task**:
> 1. Compute and visualize the correlation matrix for behavioral features.
> 2. Calculate the VIF for our behavioral markers.
> 3. Convert demographic variables into the proper categorical format.





In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Variable Selection
behavioral_cols = [
    'total_duration', 'daily_avg_duration',	'day_night_diff', 'social_media_duration',
    'games_duration', 'shopping_duration'
]

# Categorical Factor Conversion
df_model = df_final.copy()
df_model['gender'] = df_model['gender'].astype('category')
df_model['age_group'] = df_model['age_group'].astype('category')

# Correlation Heatmap
plt.figure(figsize=(10, 7))
corr_matrix = df_model[behavioral_cols].corr()
sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, fmt=".2f", vmin=-1, vmax=1)
plt.title("Correlation Matrix: Behavioral Predictors")
plt.show()

# VIF Calculation
# We add a constant as VIF requires an intercept to be mathematically sound
X_vif = df_model[behavioral_cols].assign(const=1)

vif_df = pd.DataFrame()
vif_df["Feature"] = X_vif.columns
vif_df["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

print("\n[VIF Diagnostics Table]")
# We exclude the 'const' row for the final display
display(vif_df[vif_df['Feature'] != 'const'].round(2).sort_values(by="VIF", ascending=False))



In [ ]:
df_model = df_model.drop(columns=['daily_avg_duration'])
behavioral_cols = [
    'total_duration',	'day_night_diff', 'social_media_duration',
    'games_duration', 'shopping_duration'
]

# 5. Final Data Integrity Check
print("\n[Descriptive Statistics: Behavioral Metrics (Hours)]")
display(df_model[behavioral_cols].describe().round(2))

### 4.3 Feature Transformation: To Scale or Not to Scale?

In this lab, we have chosen to keep our behavioral predictors in their **raw units (hours)** to ensure that our coefficients ($\beta$) have a direct physical interpretation (e.g., "one extra hour of gaming leads to $X$ points change in stress"). However, this choice involves a trade-off regarding **Model Convergence**.

Unlike simple OLS regression, LMMs are estimated using iterative numerical algorithms (such as **Restricted Maximum Likelihood - REML**). The algorithm "searches" for the best parameters that maximize the likelihood of the data.
* **The Convergence Risk**: If our predictors have vastly different scales (e.g., one variable ranges from 0.1 to 1.0, while another ranges from 10 to 500), the optimization surface becomes very "steep" in some directions and "flat" in others. This can cause the algorithm to fail, resulting in a **"Convergence Warning"** or a **"Singular Fit"**.
* **Standardization as a Solution**: Standardizing variables (Z-score) ensures all predictors have a mean of 0 and a standard deviation of 1. This "round" optimization space makes it much easier for the algorithm to reach the global optimum.

> **Instruction**: We will proceed with raw hours. If you encounter a `ConvergenceWarning` or `Singular Matrix` error in the next step, the first troubleshooting step should be returning here to standardize your features.

## 5. Linear Mixed-Effects Modeling (LMM)

In this section, we move beyond simple correlation to **statistical modeling**. Our data has a nested structure: multiple observations (Waves) are clustered within individuals (`pid`).

### 5.1 Why use LMM?
A standard linear regression (OLS) assumes all observations are independent. However, a person's loneliness score at Wave 2 is likely related to their score at Wave 1. LMM accounts for this by splitting the model into:
1.  **Fixed Effects**: The average relationship between digital behavior and mental health across the whole population.
2.  **Random Effects**: The individual "baseline" differences. Some people are naturally more lonely or stressed than others.

### 5.2 Model Specification
We use the following formula for both models:
$$MentalHealth_{ij} = (\beta_0 + u_i) + \beta_1(Behavior_{ij}) + \beta_2(Demographics_i) + \epsilon_{ij}$$

Where:
* $u_i$ is the **Random Intercept** for person $i$ (capturing individual stability).
* $\beta$ are the **Fixed Effects** (the universal trends we want to discover).


---

## 5.3 Modeling Loneliness (UCLA-3)

In this step, we evaluate not just the coefficients, but also the **Model Performance**:
* **ICC (Intraclass Correlation Coefficient)**: Proportion of variance explained by individual differences.
> ***Rule of Thumb**: If **$ICC > 0.05$**, it indicates that the nested structure of the data is significant, and using an LMM is mandatory to avoid biased results.*
* **Marginal $R^2$**: Variance explained by our behavioral predictors (Fixed Effects).
* **Conditional $R^2$**: Total variance explained by the entire model (Fixed + Random Effects).

> **Task**: Fit the Loneliness model and calculate these model performance index.

<mark>**Question 1**: Is loneliness related to total online time?

In [ ]:
def fit_mixed_model(df, formula, group_col='pid'):
    """
    Fits a Linear Mixed-Effects Model and attaches ICC, Marginal R2,
    and Conditional R2 as attributes to the returned model object.
    """
    # 1. Fit the Model
    model = smf.mixedlm(formula, df, groups=df[group_col]).fit()

    # 2. Extract Variance Components
    # var_random: Intercept variance (Between-person)
    # var_resid: Residual variance (Within-person / Error)
    var_random = model.cov_re.iloc[0, 0]
    var_resid = model.scale

    # 3. Calculate Fixed Effects Variance
    fixed_preds = model.predict(df)
    var_fixed = fixed_preds.var()

    # 4. Calculate Metrics
    total_var = var_fixed + var_random + var_resid

    # Attach as attributes to the model object
    model.icc = var_random / (var_random + var_resid)
    model.marginal_r2 = var_fixed / total_var
    model.conditional_r2 = (var_fixed + var_random) / total_var

    # 5. Print Summary Table
    print("\n" + "="*45)
    print(f"{'MIXED MODEL DIAGNOSTICS':^45}")
    print("-" * 45)
    print(f"Marginal R2 (Fixed Effects):    {model.marginal_r2:.4f}")
    print(f"Conditional R2 (Total Model):   {model.conditional_r2:.4f}")
    print(f"ICC (Individual Level):         {model.icc:.4f}")
    print("="*45 + "\n")

    return model


In [ ]:
formula = "ucla_3 ~ total_duration + gender + age_group"
model_1 = fit_mixed_model(df_model, formula )
print(model_1.summary())

---
## Project: Behavioral Traces & Well-being


Discover new insights from the digital trace data.

### Selection:
* **Question**: Are you predicting **Stress** (`pss_10`) or **Loneliness** (`ucla_3`)?
* **Hypothesis**: Which behavior do you think matters?
    * *Option A*: Use existing features (`day_night_diff`, `social_media_duration`, etc.).
    * *Option B*: Go back to **Section 3.1** and define a new feature.
* **Check for Issues**: If you use multiple behavioral features, reuse the **correlation & VIF code** to check for multi-collinearity.

### Modeling & Analysis
* Build at least **two different models** to compare.
* Sub-group Analysis: Does your hypothesis hold for everyone? Try filtering the data:
    ```python
    # Example: Running the model only for the 'Younger' group (age_group == 1)
    df_young = df_model[df_model['age_group'] == 1]
    model_young = smf.mixedlm("pss_10 ~ social_media_duration", df_young, groups=df_young["pid"]).fit()
    ```

### Share results
* Use the `plot_effect_comparison` function to visualize your models.
* **what** the coefficients tell us.

### Visualizing Model Findings: Effect Size Comparison

In the final step of our analysis, we will visualize the **Fixed Effects** from our LMMs.
### Interpreting the Plot
* **X-axis (Coefficient)**: Represents the change in mental health score for every **1-hour increase** in behavior.
* **Positive Values**: The behavior is associated with *increased* loneliness/stress.
* **Vertical Line at 0**: If the error bar crosses this line, the effect is likely not statistically significant.


In [ ]:
# ==========================================
# WORKSPACE
# ==========================================

#-----Define and fit your models-----
# model_a = fit_mixed_model(df, formula, group_col='pid')
# model_b = fit_mixed_model(df, formula, group_col='pid')

#print(model_a.summary())

In [ ]:
def plot_behavioral_comparison(model_dict, behavioral_features=None):
    """
    Plots a grouped bar chart for multiple models with significance stars.

    Args:
        model_dict (dict): Format {'Display Name': model_object}
        behavioral_features (list): List of predictor names to include in the plot.
    """

    # Default feature list if none provided
    if behavioral_features is None:
        behavioral_features = ['social_media_duration', 'games_duration', 'shopping_duration', 'day_night_diff']

    # 1. Consolidate results using the dictionary keys as 'Model' labels
    all_results = []
    for label, model_obj in model_dict.items():
        temp_df = pd.DataFrame({
            'Predictor': model_obj.params.index,
            'Coefficient': model_obj.params.values,
            'p_value': model_obj.pvalues.values,
            'Model': label
        })
        # Filter only for the features we care about
        temp_df = temp_df[temp_df['Predictor'].isin(behavioral_features)].copy()
        all_results.append(temp_df)

    results = pd.concat(all_results).reset_index(drop=True)

    # 2. Dynamic Plot Sizing
    # Adjust height based on number of models and predictors
    num_models = len(model_dict)
    num_vars = len(behavioral_features)
    plt.figure(figsize=(14, max(6, num_vars * num_models * 0.5)))

    # 3. Initialize the plotting area
    # Use 'order' to ensure the Y-axis follows your input list exactly
    ax = sns.barplot(
        data=results,
        x='Coefficient',
        y='Predictor',
        hue='Model',
        palette="RdYlBu",
        order=behavioral_features
    )

    # 4. Add Significance Stars (*)
    # We iterate through containers (one for each model/hue)
    for i, container in enumerate(ax.containers):
        # Get the label for the current hue group
        model_name = list(model_dict.keys())[i]
        subset = results[results['Model'] == model_name].set_index('Predictor')

        # Annotate each bar in the current container
        for j, bar in enumerate(container):
            # Map bar back to predictor via the fixed Y-axis order
            predictor_name = behavioral_features[j]

            if predictor_name in subset.index:
                p_val = subset.loc[predictor_name, 'p_value']

                if p_val < 0.05:
                    width = bar.get_width()
                    y_pos = bar.get_y() + bar.get_height() / 2

                    # Strategic offset: 1% of the x-axis range
                    x_range = ax.get_xlim()[1] - ax.get_xlim()[0]
                    offset = 0.01 * x_range if width >= 0 else -0.01 * x_range

                    ax.text(
                        width + offset, y_pos, '*',
                        ha='center', va='center',
                        fontsize=20, color='black', fontweight='bold'
                    )

    # 5. Aesthetics and Labeling
    plt.axvline(0, color='black', linestyle='-', linewidth=1.5, alpha=0.5)
    plt.title("Impact of Digital Behaviors on Mental Health\n(Stars * indicate p < 0.05)", fontsize=16, pad=20)
    plt.xlabel("Effect Size (Coefficient Beta)", fontsize=12)
    plt.ylabel("Digital Behavior Predictors", fontsize=12)

    # Place legend outside
    plt.legend(title="Model / Group", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(axis='x', linestyle=':', alpha=0.6)

    plt.tight_layout()
    plt.show()



In [ ]:
# ----- Effect Size Comparison
# features will included
my_features = [
    'total_duration',
]

# 2.  models in a dictionary (supports 2 to 8 models)
my_models = {
    'Loneliness (Total)': model_1,
}

# 3. Run the function
plot_behavioral_comparison(my_models, behavioral_features=my_features)